In [ ]:
# !pip install keras-tuner

# California Housing Dataset

In [2]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError




In [6]:
#Train file
df_train = pd.read_csv('D:\\lh_data\\ineuron\\california_housing_train.csv')

#Test file
df_test = pd.read_csv('D:\\lh_data\\ineuron\\california_housing_test.csv')

In [7]:
df_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [8]:
df_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0


In [10]:
#shape of train and test
print(df_train.shape)
print(df_test.shape)

(17000, 9)
(3000, 9)


In [12]:
df_train.describe().T    

,count,mean,std,min,25%,50%,75%,max
longitude,17000.0,-119.562108,2.005166,-124.3500,-121.790000,-118.4900,-118.000,-114.3100
latitude,17000.0,35.625225,2.137340,32.5400,33.930000,34.2500,37.720,41.9500
housing_median_age,17000.0,28.589353,12.586937,1.0000,18.000000,29.0000,37.000,52.0000
total_rooms,17000.0,2643.664412,2179.947071,2.0000,1462.000000,2127.0000,3151.250,37937.0000
total_bedrooms,17000.0,539.410824,421.499452,1.0000,297.000000,434.0000,648.250,6445.0000
population,17000.0,1429.573941,1147.852959,3.0000,790.000000,1167.0000,1721.000,35682.0000
households,17000.0,501.221941,384.520841,1.0000,282.000000,409.0000,605.250,6082.0000
median_income,17000.0,3.883578,1.908157,0.4999,2.566375,3.5446,4.767,15.0001
median_house_value,17000.0,207300.912353,115983.764387,14999.0000,119400.000000,180400.0000,265000.000,500001.0000


In [13]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,3000.0,-119.589200,1.994936,-124.1800,-121.810,-118.48500,-118.020000,-114.4900
latitude,3000.0,35.635390,2.129670,32.5600,33.930,34.27000,37.690000,41.9200
housing_median_age,3000.0,28.845333,12.555396,1.0000,18.000,29.00000,37.000000,52.0000
total_rooms,3000.0,2599.578667,2155.593332,6.0000,1401.000,2106.00000,3129.000000,30450.0000
total_bedrooms,3000.0,529.950667,415.654368,2.0000,291.000,437.00000,636.000000,5419.0000
population,3000.0,1402.798667,1030.543012,5.0000,780.000,1155.00000,1742.750000,11935.0000
households,3000.0,489.912000,365.422710,2.0000,273.000,409.50000,597.250000,4930.0000
median_income,3000.0,3.807272,1.854512,0.4999,2.544,3.48715,4.656475,15.0001
median_house_value,3000.0,205846.275000,113119.687470,22500.0000,121200.000,177650.00000,263975.000000,500001.0000


* the above distributins are very similar

In [14]:
df_train.isna().sum()   # no null values found

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [15]:
df_test.isna().sum()   # no null values found

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [26]:
# split the data into dependent and independent

TARGET_NAME = 'median_house_value'

x_train, y_train = df_train.drop(TARGET_NAME, axis=1), df_train[TARGET_NAME]
x_test, y_test = df_test.drop(TARGET_NAME, axis=1), df_test[TARGET_NAME]

x_train.shape, y_train.shape, x_test.shape, y_test.shape


((17000, 8), (17000,), (3000, 8), (3000,))

In [27]:
print("*"*20 , "Before Scaling" ,"*"*20)
x_train.describe().T  


******************** Before Scaling ********************


,count,mean,std,min,25%,50%,75%,max
longitude,17000.0,-119.562108,2.005166,-124.3500,-121.790000,-118.4900,-118.000,-114.3100
latitude,17000.0,35.625225,2.137340,32.5400,33.930000,34.2500,37.720,41.9500
housing_median_age,17000.0,28.589353,12.586937,1.0000,18.000000,29.0000,37.000,52.0000
total_rooms,17000.0,2643.664412,2179.947071,2.0000,1462.000000,2127.0000,3151.250,37937.0000
total_bedrooms,17000.0,539.410824,421.499452,1.0000,297.000000,434.0000,648.250,6445.0000
population,17000.0,1429.573941,1147.852959,3.0000,790.000000,1167.0000,1721.000,35682.0000
households,17000.0,501.221941,384.520841,1.0000,282.000000,409.0000,605.250,6082.0000
median_income,17000.0,3.883578,1.908157,0.4999,2.566375,3.5446,4.767,15.0001


In [30]:
# Feature Scaling
sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.transform(x_test)

In [34]:
print("*"*20 , "After Scaling" ,"*"*20)
pd.DataFrame(x_train_sc, columns=x_train.columns).describe().T

******************** After Scaling ********************


,count,mean,std,min,25%,50%,75%,max
longitude,17000.0,-9.134288e-15,1.000029,-2.387848,-1.111108,0.534689,0.779065,2.619365
latitude,17000.0,-1.540258e-15,1.000029,-1.443531,-0.793170,-0.643447,0.980114,2.959268
housing_median_age,17000.0,-9.654238e-16,1.000029,-2.191968,-0.841322,0.032626,0.668224,1.859971
total_rooms,17000.0,7.481924e-17,1.000029,-1.211838,-0.542077,-0.237015,0.232850,16.190473
total_bedrooms,17000.0,1.571553e-16,1.000029,-1.277408,-0.575132,-0.250093,0.258227,14.011318
population,17000.0,2.034190e-16,1.000029,-1.242856,-0.557208,-0.228759,0.253895,29.841308
households,17000.0,-5.476012e-17,1.000029,-1.300935,-0.570134,-0.239843,0.270547,14.514017
median_income,17000.0,-3.983643e-16,1.000029,-1.773323,-0.690322,-0.177652,0.462985,5.825963


##Keras Tuner

An open-source Python library called Keras Tuner was created specifically for adjusting the hyperparameters of artificial neural networks. Currently, the Keras tuner supports four different types of tuners or algorithms:

* Bayesian Optimization
* Hyperband
* Sklearn
* Random Search

Using the model-building feature we can enable this tuning feature
We are going to build the ann with 4 layers

1. Input layer
2. Hidden layer 1      unit1 will have the range between 25 to 512 with step 32
3. Hidden layer 2      unit2 will have the range between 25 to 512 with step 32
4. Output Layer

These will be apply to input layer and hidden layers

The learning rate will be picked up based on given inputs and hence we are use choice method to perform that action

Since this is the regression problem, we are using 'linear' activation function in the output layer. on the hidden layers we are using 'relu'.

For weight updation, we are using adam optimizers and the loss will be MSLE (Mean Squarred Log Error)

In [35]:
x_train_sc.shape

(17000, 8)

In [53]:
msle = MeanSquaredLogarithmicError()


def build_annmodel(hp):
    model = tf.keras.Sequential()
  
    units1 = hp.Int('units1', min_value=25, max_value=512, step=32)
    units2 = hp.Int('units2', min_value=25, max_value=512, step=32)
    units3 = hp.Int('units3', min_value=25, max_value=512, step=32)
    model.add(Dense(units=units1, input_shape=(8,),activation="relu"))  # adds the input layer with 8 neurons, one for each input features 
    model.add(tf.keras.layers.Dense(units=units2, activation='relu'))
    model.add(tf.keras.layers.Dense(units=units3, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss=msle, metrics=[msle])
    
    model.summary()

    return model

* In order to get a high-performing model quickly, the Hyperband tuning algorithm makes use of adaptive resource allocation and early-stopping.

In [54]:
# HyperBand algorithm from keras tuner

tuner = kt.Hyperband(
    build_annmodel,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 25)                225       
_________________________________________________________________
dense_9 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_10 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 26        
Total params: 1,551
Trainable params: 1,551
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Create a callback to stop training early after reaching a certain value for the validation loss.

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [56]:
tuner.search(x_train_sc, y_train, epochs=20, validation_split=0.2)

Trial 30 Complete [00h 00m 09s]
val_mean_squared_logarithmic_error: 0.3557300865650177

Best val_mean_squared_logarithmic_error So Far: 0.1128627210855484
Total elapsed time: 00h 02m 06s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [57]:
tuner.search(x_train_sc, y_train, epochs=20, validation_split=0.2, callbacks=[stop_early])

Trial 42 Complete [00h 00m 09s]
val_mean_squared_logarithmic_error: 0.44542112946510315

Best val_mean_squared_logarithmic_error So Far: 0.10973088443279266
Total elapsed time: 00h 04m 17s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [58]:
# Display search space summary

tuner.search_space_summary()

Search space summary
Default search space size: 4
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 512, 'step': 32, 'sampling': None}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 512, 'step': 32, 'sampling': None}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [59]:
# You can also view the optimal hyperparameter values found by the search:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps
print(f"""
The hyperparameter search is complete. 
The optimal number of units in the densed layers are: units1:{best_hps.get('units1')}, units2:{best_hps.get('units2')}, units3:{best_hps.get('units3')}  
The optimal learning rate for the optimizer is: {best_hps.get('learning_rate')}. """)


The hyperparameter search is complete. 
The optimal number of units in the densed layers are: units1:249, units2:89, units3:89  
The optimal learning rate for the optimizer is: 0.01. 


In [60]:
# can be also displayed this way:

for p in ['units1','units2','units3','learning_rate']:
    print(p, tuner.get_best_hyperparameters()[0].get(p))

units1 249
units2 89
units3 89
learning_rate 0.01


In [43]:
tuner.get_best_hyperparameters(num_trials=1)[0]

# Model Evaluation

In [61]:
model = tuner.hypermodel.build(best_hps)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 249)               2241      
_________________________________________________________________
dense_5 (Dense)              (None, 89)                22250     
_________________________________________________________________
dense_6 (Dense)              (None, 89)                8010      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 90        
Total params: 32,591
Trainable params: 32,591
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs

model = tuner.hypermodel.build(best_hps)   # creates hypermodel with the best hyper parameters

history = model.fit(x_train_sc, y_train, epochs=50, validation_split=0.2)



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 249)               2241      
_________________________________________________________________
dense_13 (Dense)             (None, 89)                22250     
_________________________________________________________________
dense_14 (Dense)             (None, 89)                8010      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 90        
Total params: 32,591
Trainable params: 32,591
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
425/425 [==============================] - 1s 2ms/step - loss: 17.5297 - mean_squared_logarithmic_error: 17.5297 - val_loss: 0.6605 - val_mean_squared_logarithmic_error: 0.6581
Epoch 2/50
425/425 [==============================] - 1s 2ms

In [64]:
history.history.keys()

dict_keys(['loss', 'mean_squared_logarithmic_error', 'val_loss', 'val_mean_squared_logarithmic_error'])

In [69]:
# best epoch

val_acc_per_epoch = history.history['val_mean_squared_logarithmic_error']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 30


In [70]:
eval_result = model.evaluate(x_test_sc, y_test)
print("[test loss, test mean_squared_logarithmic_error]:", eval_result)

94/94 [==============================] - 0s 763us/step - loss: 0.0795 - mean_squared_logarithmic_error: 0.0794
[test loss, test mean_squared_logarithmic_error]: [0.0794663354754448, 0.07942277938127518]


In [71]:
eval_result

[0.0794663354754448, 0.07942277938127518]

In [72]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 249)               2241      
_________________________________________________________________
dense_13 (Dense)             (None, 89)                22250     
_________________________________________________________________
dense_14 (Dense)             (None, 89)                8010      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 90        
Total params: 32,591
Trainable params: 32,591
Non-trainable params: 0
_________________________________________________________________
